In [1]:
%matplotlib inline

import scipy.io as scio
import SimpleITK as sitk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from myshow import myshow, myshow3d

In [11]:
img = sitk.ReadImage('liver2.nii.gz')
img_255 = sitk.Cast(sitk.RescaleIntensity(img), sitk.sitkUInt8)
myshow3d(img)

interactive(children=(IntSlider(value=229, description='z', max=459), Output()), _dom_classes=('widget-interac…

In [3]:
seed_pts = [(240,195,211), (310,220,211)]
seg = sitk.Image(img.GetSize(), sitk.sitkUInt8)
seg.CopyInformation(img)
seg[seed_pts[0]] = 1
seg[seed_pts[1]] = 1
seg = sitk.BinaryDilate(seg, [3]*seg.GetDimension())
myshow(sitk.LabelOverlay(img_255, seg), "Initial Seed")

interactive(children=(IntSlider(value=229, description='z', max=459), Output()), _dom_classes=('widget-interac…

In [4]:
seg = sitk.ConfidenceConnected(img, seedList=seed_pts,
                                   numberOfIterations=1,
                                   multiplier=4,
                                   initialNeighborhoodRadius=1,
                                   replaceValue=1)

In [6]:
myshow(sitk.LabelOverlay(img_255, seg), "ConfidenceConnected")

interactive(children=(IntSlider(value=229, description='z', max=459), Output()), _dom_classes=('widget-interac…

In [9]:
BMC = sitk.BinaryMorphologicalClosingImageFilter()
BMC.SetKernelType(sitk.sitkBall)
BMC.SetKernelRadius(1)
BMC.SetForegroundValue(1)
OUT = BMC.Execute(seg)

In [10]:
myshow(sitk.LabelOverlay(img_255, OUT), "BMClosingImage")

interactive(children=(IntSlider(value=229, description='z', max=459), Output()), _dom_classes=('widget-interac…

In [8]:
OUT_225 = OUT[:, :, :397]
out_erode = sitk.BinaryErode(OUT_225, [4] * OUT_225.GetDimension())
out_dilate = sitk.BinaryDilate(out_erode, [4] * out_erode.GetDimension())
myshow(sitk.LabelOverlay(img_255[:, :, :397], out_dilate), "dilated")

interactive(children=(IntSlider(value=198, description='z', max=396), Output()), _dom_classes=('widget-interac…

In [9]:
out_final = sitk.Paste(OUT, out_dilate, out_dilate.GetSize(), destinationIndex=[0,0,0])
myshow(sitk.LabelOverlay(img_255, out_final), "ConfidenceConnected")

interactive(children=(IntSlider(value=229, description='z', max=459), Output()), _dom_classes=('widget-interac…

In [7]:
out_array = sitk.GetArrayFromImage(OUT)
origin = OUT.GetOrigin()
direction = OUT.GetDirection()
space = OUT.GetSpacing()

In [41]:
savedImg = sitk.GetImageFromArray(out_array)
savedImg.SetOrigin(origin)
savedImg.SetDirection(direction)
savedImg.SetSpacing(space)
sitk.WriteImage(savedImg, 'spleen_seg2.nii.gz')